# HW3 - Q-learning and Actor Critic
## Q1 DQN:
Failed to complete because I get this error in dqn_utils:

```x = x.permute(0, 3, 1, 2).contiguous()
RuntimeError: number of dims don't match in permute```

## Q2  Double DQN - Lunar Lander:
Double DQN looks slightly better on average (highest return run is ddqn and lowest return is dqn). However, the behavior is mostly affected by the random seed used. The theory behind double dqn is that the target Q-val is evaluated with the max of target Q network but using action chosen from active Q-network. This decorrelation within the max operator avoids network overestimating value fn.

<img src="q2.png" width="600"/>

### Double DQN Pseudo-code

Example from Q2 Lunar Lander

---
**for** n_iter (e.g. 500,000)
<blockquote>
    Take single step in environment:
    <blockquote>
        - Sample action with $\epsilon$-greedy, with $\epsilon$ prob of random action <br>
        - else, use argmax policy from current Q-fn:
        <blockquote>
            - get Q-value for each action $qa\_vals = Q(s)$<br>
            - select action with highest reward-to-go: $a=argmax(qa\_vals)$
        </blockquote>
        - Take single step in env given action<br>
        - Store the single transition: $s$, $a$, $r$, $s'$ in replay buffer
    </blockquote>
    Train DQN "Critic" network:
    <blockquote>
        - Sample `train_batch_size` (32) transitions randomly from replay buffer<br>
        - Compute targets (non-diff): $y = r + \gamma Q_{t}(s', argmax_{a'}(Q(s'))$ (double DQN)<br>
        - Compute return estimate from original action: $Q(s,a) = argmax_a(Q(s))$<br>
        - Compute loss: $loss = (y - Q(s,a))^2$<br>
        - Backprop grad and 1 grad step<br>
        - Update $Q_t$ target network params to copy $Q$ network every `target_update_freq` cycles (3,000)<br>
    </blockquote>
</blockquote>

**end for**

---
where:
- critic network ($Q(s)$ and $Q_t(s)$) has 9 input nodes (# states) with 1 hidden layer with 64 nodes with `ReLU()` activation, and 6 output (# discrete actions) representing return-to-go estimate for each action. 

## Q3 Double DQN Hyperparams:

- blue: greedy argmax policy
- gray: $\epsilon$-greedy policy where random actions are taken with prob $\epsilon$ which decreases over iterations
- orange: $\epsilon$-greedy where first 100k iterations are taken with random actions ($\epsilon$ = 1), then $\epsilon$ decreases over rest of iters

The additional exploration of epsilon-greedy improved avg return as seen below. This makes sense as it avoid argmax policy overfitting to initial inaccurate Q-fn. Even when random actions are taken for first 100k iterations, the returns are still able to catchup to trial with no epsilon-greedy.

<img src="q3.png" width="600"/>

## Q4 Actor-Critic Cart-Pole:
<img src="q4.png" width="600"/>

params:

```--env_name CartPole-v0 -n 100 -b 1000 --exp_name q4_ac_10_10 -ntu 10 -ngsptu 10 -ngpu --video_log_freq -1```

## Q5 More Actor-Critic :
| Inv. Pendulum | Half Cheetah |
| :---: | :---: |
| <img src="inv_pend_AC.gif" width="400"/> | <img src="cheetah_AC.gif" width="400"/> |
| <img src="q5_AC_inv_pend.png" width="400"/> | <img src="q5_AC_cheetah.png" width="400"/> |

params:

```--env_name InvertedPendulum-v2 --ep_len 1000 --discount 0.95 -n 100 -l 2 -s 64 -b 5000 -lr 0.01 --exp_name q5_10_10 -ntu 10 -ngsptu 10 -ngpu --video_log_freq 50```

```--env_name HalfCheetah-v2 --ep_len 150 --discount 0.90 --scalar_log_freq 1 -n 150 -l 2 -s 32 -b 30000 -eb 1500 -lr 0.02 --exp_name q5_10_10 -ntu 10 -ngsptu 10 -ngpu --video_log_freq 50```

### Actor-Critic Pseudo-code

Example from Q5 half-cheetah

---
**for** n_iter (e.g. 150)
<blockquote>
    collect_training_trajectories with current policy
    <blockquote>
        - with batch_size (30k) env samples with ep_len (150) samples per rollout (i.e. 200 rollouts) <br>
        - sampling actions from $\pi(a|o)$ distribution
    </blockquote>    
    - sample train_batch_size (30k) most recent samples (need to be collected from curr policy due to on-policy)<br>
    - Update critic:<br>
    for num_target_updates (e.g. 10):   
    <blockquote>
        - compute targets for all batch samples: $y_t = r(s_t, a_t) + \gamma V(s_{t+1})$ (convert to np so grad not computed, ideally do this to convergence but limitted by compute)<br> 
        for num_grad_steps_per_target_update (e.g. 10):
        <blockquote>
            - $loss = (V(s)- y_t)^2/N$<br>
            - 1 grad step
        </blockquote>
    </blockquote>
    - compute advantage with critic network: $A(s, a) = Q(s, a) - V(s) = r(s,a) + \gamma V(s_{t+1}) - V(s)$<br>
    - Perform num_actor_updates_per_agent_update (e.g. 1) grad step in policy with $\sum_{t=0}^{T-1} [\nabla [log \pi(a_t|o_t) * (A(s,a))]]$<br>
    So `loss=-Sum(log_prob_of_action * advantages)` where gradient is not propagated through baseline NN used for advantage estimation<br>
    - Perform one grad step in NN baseline (value fn estimator, normalized)
</blockquote>

**end for**

---
where:
- critic network (to estimate $V(s)$) has 17 input nodes (# states) with 1 hidden layer with 32 nodes with `Tanh()` activation, and 1 output (return to go value est). 
- actor policy network ($\pi (a|s)$) also has 17 input nodes (# states) with 1 hidden layer with 32 nodes with `Tanh()` activation, and 6 output nodes (# actions)